In [ ]:
# Core implementation of turbulent heat and water continuity, we provide the main commands to compute the 
# discrete equations in Algorithms 1 and 2. 

import numpy as np

# Main parameters (check Table 2 in the manuscript)
rho_d = 2500
gass_r = 0.287
gamma0 = -6.5e-3
gamma1 = 6.5e-3
z1 = 8e3
s_k = 0.1

# Get background temperature profile
# Input:
# groundT - 2D grid with ground temperature (can be time dependent - series of 2D textures)
# idx = (i,j,k) - 3D coordinates
# gamma0, gamma1 - lapse rates
# z1 - inversion altitude
def backgroundT(groundT, idx, gamma0, gamma1, z1):
    tempGround = groundT[idx[0]][idx[1]]
    if(x[2]<=z1):
        return tempGround - gamma0*idx[2]
    else:
        return tempGround - gamma0*z1 - gamma1* (idx[2]-z1)


# Get background pressure profile
# Input:
# groundT - 2D grid with ground temperature
# x - 3D coordinates
# q_v - mixing ratio of vapor
def backgroundP(groundT, idx, q_v):
    return rho_d * gass_r * backgroundT(groundT, idx, gamma0, gamma1, z1) * (1.0+0.61*q_v)

# Get turbulent viscosity
# Input:
# k - turbulent energy field
# idx  = (i,j,k) - 3D coordinates
def nu_t(k, idx):
    return 0.2 * s_k * np.sqrt(k[idx[0]][idx[1]][idx[2]])



